# Imports

In [ ]:
# Import libraries
# !pip install --upgrade pandas
# !pip install prophet
# !pip install -U numpy
# !pip uninstall pillow
# !pip install pillow
# !pip install -U scipy
# !pip install matplotlib
# !pip install seaborn matplotlib
# !pip install statsmodels
# !pip install sklearn

import pandas as pd
from prophet import Prophet
import numpy as np
import numpy.ma as ma

from scipy import stats

import seaborn as sns
from scipy.interpolate import make_interp_spline
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from sklearn.preprocessing import StandardScaler

import warnings
import logging

from column import Column
from region import Region
from util import plotRegionObservations, findRegionsByNumberObservations, findRegionsByName, boxplotsOfDataframe, histogramsOfDataframe

# Info about our dataset

On the dataset we are going to examine we have data downloaded from the Hass Avocado Board website in May of 2018. The data represents weekly retail scanned data for National retail volume (units) and price between 2015 and 2018.

The columns of the csv file are the following:

- Unnamed column that represents the number of weeks left for the end of the year of the observation
- Date - The date of the observation
- AveragePrice - the average price of a single avocado
- Total Volume - Total number of avocados sold
- 4046 - Total number of avocados with PLU 4046 sold
- 4225 - Total number of avocados with PLU 4225 sold
- 4770 - Total number of avocados with PLU 4770 sold
- Total Bags - Total number of bags of avocados sold
- Small Bags - Total number of small bags of avocados sold
- Large Bags - Total number of large bags of avocados sold
- XLarge Bags - Total number of xlarge bags of avocados sold
- type - conventional or organic
- year - the year of the observation
- region - the city or region of the observation

In [ ]:
# Load the dataset
avocado_df = pd.read_csv('./data/avocado.csv')

avocado_df.head()

# Delete index column

As the unnmaed column that represents the number of weeks left for the end of the year of the observation does not seem of use we will remove it

In [ ]:
avocado_df = avocado_df.drop(list(avocado_df)[0], axis=1)

avocado_df.head()

# Exploratory Data Analysis

## Regions

We will examine first the regions column, its distribution and an interesting fact that we need to consider in our analysis 

In [ ]:
regions = avocado_df['region'].value_counts()

print("Number of regions on the dataset are",len(regions))

As we can see on the next histogram most of the regions/cities of our dataset have 338 observations

In [ ]:
plotRegionObservations(regions)

As we will see below the only region with 335 observations is WestTexNewMexico

In [ ]:
findRegionsByNumberObservations(regions, 335)

One of the important facts we need to consider about this dataset though is that there is a region named TotalUS

In [ ]:
findRegionsByName(regions, "TotalUS")

That means in order to examine the whole US and then the weight for each region we should not work on the whole dataset and the each different region.

We need to filter the dataframe and work with the observations that have the value TotalUS as region to understand what happens to US as a whole.

Of course once we need to examine each region on its own or compared to whole US we need to take into account for the region WestTexNewMexico that it has 3 less observations.

## Summary statistics for Total US

### Distributions of columns

We will calculate summary stats like mean, median, min, max, quantiles, etc. for each column to understand distributions. After that the next step will be to use boxplots and histograms as well.

Before doing that we will do a check to see the type of data each column holds

In [ ]:
total_us_df = avocado_df[avocado_df["region"]=="TotalUS"]

#Types of columns
print(total_us_df.dtypes)

As not every column holds numeric data (float or int) we will focus on the numeric ones

#### Summary stats

In [ ]:
# Summary stats
print(total_us_df.describe())

#### Boxplots

In [ ]:
# boxplotsOfDataframe(total_us_df)

#### Histograms

In [ ]:
total_us_df = total_us_df.sort_values(by='Date')
total_us_df.reset_index(drop=True)

total_us_df.head()

In [ ]:
histogramsOfDataframe(total_us_df)

## Analyze trends over time - create line charts for average price, volume, bags sold, etc. over the date range to visually inspect trends and seasonality.

### Average Price trends

To examine the trends for average price we are going to plot a graph for each year and then try to compare the years to see obvious patterns based on seasonality.

First step is to create a new dataframe that will combine the duplicate rows for the same date as the current dataset has one row for each date, one for the organic avocados observation and the other for the conventional. Then this new dataframe will be used to plot our graphs.

In [ ]:
# Pivot the DataFrame to have 'Date' as the index and 'Type' as columns
pivot_df = total_us_df.pivot(index=['Date','year'], columns='type', values='AveragePrice')

# Calculate the average of 'Organic' and 'Conventional' columns
pivot_df['AveragePriceCombined'] = pivot_df.mean(axis=1)

# Reset the index to have 'Date' as a regular column
result_df = pivot_df.reset_index()

# Optionally, you can rename the columns as needed
result_df.rename(columns={'Date': 'Date', 'AveragePriceCombined': 'AveragePrice','organic':'Organic','conventional':'Conventional'}, inplace=True)

The graphs will have 3 lines. One line will be the average price of the organic avocados through the year, the other for the conventional and the final one will be the combination of the first two.

In [ ]:
years_df = result_df.loc[:]
# [result_df['year']==year]
years_df.reset_index()

# Create a line plot
plt.figure(figsize=(15, 6))
for year in range(2015,2017):
    plt.plot(years_df[years_df['year']==year]['Date'], years_df[years_df['year']==year]['Organic'], label=f'Organic_{year}')
    plt.plot(years_df[years_df['year']==year]['Date'], years_df[years_df['year']==year]['Conventional'], label=f'Conventional_{year}')
    # plt.plot(year_df['Date'], year_df['AveragePrice'], label='Average Price', linestyle='--')

# Add labels and a legend
plt.xlabel('Date')
plt.ylabel('Average Price')
plt.title('Price Trend for Conventional and Organic avocados')
plt.legend()

# Rotate x-axis labels for better readability (optional)
plt.xticks(rotation=45)

# Show the plot
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
years_df[years_df['year']==2015]['Date'].head()

In [ ]:
years_df = result_df.loc[:]
years_df['Date'] = pd.to_datetime(years_df['Date'])
years_df['Date'] = years_df['Date'].dt.month_name() + ' ' + years_df['Date'].dt.day.astype(str)
# years_df['Date'] = pd.to_datetime(years_df['Date'])
years_df.head()

In [ ]:
years_df = result_df.loc[:]
years_df.reset_index()

years_df['Date'] = pd.to_datetime(years_df['Date'])
years_df['Date'] = years_df['Date'].dt.month_name() + ' ' + years_df['Date'].dt.day.astype(str)
# years_df['Date'] = pd.to_datetime(years_df['Date'])
# [result_df['year']==year]

# Create a line plot
plt.figure(figsize=(15, 6))
for year in range(2015,2018):
    plt.plot(years_df[years_df['year']==year]['Date'], years_df[years_df['year']==year]['Organic'], label=f'Organic_{year}')
    plt.plot(years_df[years_df['year']==year]['Date'], years_df[years_df['year']==year]['Conventional'], label=f'Conventional_{year}')
    # plt.plot(year_df['Date'], year_df['AveragePrice'], label='Average Price', linestyle='--')

# Add labels and a legend
plt.xlabel('Date')
plt.ylabel('Average Price')
plt.title('Price Trend for Conventional and Organic avocados')
plt.legend()

# Rotate x-axis labels for better readability (optional)
plt.xticks(rotation=45)

# Show the plot
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate daily price difference  
result_df['PriceDiffOrganic'] = result_df['Organic'].diff()
result_df['PriceDiffConventional'] = result_df['Conventional'].diff()

# Calculate percentage difference
result_df['PriceDiffPercentOrganic'] = result_df['PriceDiffOrganic']/result_df['Organic'].shift(1) * 100
result_df['PriceDiffPercentConventional'] = result_df['PriceDiffConventional']/result_df['Conventional'].shift(1) * 100

# # Plot average price over date
plt.figure(figsize=(24,8))
# Plot trends
# plt.plot(result_df['Date'], result_df['PriceDiff'], label='Daily Difference')
plt.plot(result_df['Date'], result_df['PriceDiffPercentOrganic'], label='Organic Daily % Change')
plt.plot(result_df['Date'], result_df['PriceDiffPercentConventional'], label='Conventional Daily % Change')

plt.legend()
plt.ylabel('USD Price Change')
plt.title('Avocado Price Trends')

In [ ]:
result_df

In [ ]:
# Calculate daily price difference  
result_df['PriceDiffOrganic'] = result_df['Organic'].diff()
result_df['PriceDiffConventional'] = result_df['Conventional'].diff()

# # Calculate percentage difference
# result_df['PriceDiffPercentOrganic'] = result_df['PriceDiffOrganic']/result_df['Organic'].shift(1) * 100
# result_df['PriceDiffPercentConventional'] = result_df['PriceDiffConventional']/result_df['Conventional'].shift(1) * 100

# # Plot average price over date
plt.figure(figsize=(24,8))
# Plot trends
# plt.plot(result_df['Date'], result_df['PriceDiff'], label='Daily Difference')
plt.plot(result_df['Date'], result_df['PriceDiffOrganic'], label='Organic Daily % Change')
plt.plot(result_df['Date'], result_df['PriceDiffConventional'], label='Conventional Daily % Change')

plt.legend()
plt.ylabel('USD Price Change')
plt.title('Avocado Price Trends')

In [ ]:
# years_df = result_df.loc[:]
# [result_df['year']==year]
years_df.reset_index()

# Create a line plot
plt.figure(figsize=(15, 6))
for year in range(2015,2017):
    plt.plot(years_df[years_df['year']==year]['Date'], years_df[years_df['year']==year]['Organic'], label=f'Organic_{year}')
    plt.plot(years_df[years_df['year']==year]['Date'], years_df[years_df['year']==year]['Conventional'], label=f'Conventional_{year}')
    # plt.plot(year_df['Date'], year_df['AveragePrice'], label='Average Price', linestyle='--')

# Add labels and a legend
plt.xlabel('Date')
plt.ylabel('Average Price')
plt.title('Price Trend for Conventional and Organic avocados')
plt.legend()

# Rotate x-axis labels for better readability (optional)
plt.xticks(rotation=45)

# Show the plot
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
for year in range(2015,2018):
    year_df = result_df[result_df['year']==year].loc[:]
    
    year_df.reset_index()
    # Create a line plot
    plt.figure(figsize=(15, 6))
    plt.plot(year_df['Date'], year_df['Organic'], label='Organic')
    plt.plot(year_df['Date'], year_df['Conventional'], label='Conventional')
    # plt.plot(year_df['Date'], year_df['AveragePrice'], label='Average Price', linestyle='--')

    # Add labels and a legend
    plt.xlabel('Date')
    plt.ylabel('Average Price')
    plt.title('Price Trend for Conventional and Organic avocados')
    plt.legend()

    # Rotate x-axis labels for better readability (optional)
    plt.xticks(rotation=45)

    # Show the plot
    plt.grid(True)
    plt.tight_layout()
    plt.show()

One of the first observations from the graphs above is that it seems there is a peak between August and October for the prices of Avocados for both the organic and conventional types. From October and till the end of the year there is a decline on the Average Price.

To understand if this decline is driven by less demand we will plot the graphs for Total Volume of avocados being sold in the whole US like we just did for the Average Price.

### Total volume trends

As the difference between volume of conventional and organic is big we will plot them in different graphs to avoid seeing a flat line for the total volume of organic.

In [ ]:
# Pivot the DataFrame to have 'Date' as the index and 'Type' as columns
pivot_df = total_us_df.pivot(index=['Date','year'], columns='type', values='Total Volume')

# Calculate the average of 'Organic' and 'Conventional' columns
pivot_df['TotalVolumeCombined'] = pivot_df.mean(axis=1)

# Reset the index to have 'Date' as a regular column
result_df = pivot_df.reset_index()

# Optionally, you can rename the columns as needed
result_df.rename(columns={'Date': 'Date', 'TotalVolumeCombined': 'TotalVolume','organic':'Organic','conventional':'Conventional'}, inplace=True)

In [ ]:
for year in range(2015,2018):
    ## Organic
    year_df = result_df[result_df['year']==year].loc[:]

    year_df.reset_index()
    # Create a line plot
    plt.figure(figsize=(15, 6))
    plt.plot(year_df['Date'], year_df['Organic'], label='Organic')
    # plt.plot(year_df['Date'], year_df['Conventional'], label='Conventional')
    # plt.plot(year_df['Date'], year_df['TotalVolume'], label='Total Volume', linestyle='--')

    # Add labels and a legend
    plt.xlabel('Date')
    plt.ylabel('Total Volume')
    plt.title(f'Total volume trend of Organic avocados for year {year}')
    plt.legend()

    # Rotate x-axis labels for better readability (optional)
    plt.xticks(rotation=45)

    # Show the plot
    plt.grid(True)
    plt.tight_layout()
    plt.show()


    ## Conventional
    year_df = result_df[result_df['year']==year].loc[:]

    year_df.reset_index()
    # Create a line plot
    plt.figure(figsize=(15, 6))
    # plt.plot(year_df['Date'], year_df['Organic'], label='Organic')
    plt.plot(year_df['Date'], year_df['Conventional'], label='Conventional')
    # plt.plot(year_df['Date'], year_df['TotalVolume'], label='Total Volume', linestyle='--')

    # Add labels and a legend
    plt.xlabel('Date')
    plt.ylabel('Total Volume')
    plt.title(f'Total volume trend of Conventional avocados for year {year}')
    plt.legend()

    # Rotate x-axis labels for better readability (optional)
    plt.xticks(rotation=45)

    # Show the plot
    plt.grid(True)
    plt.tight_layout()
    plt.show()

## Correlation analysis - calculate correlation coefficients between variables to identify relationships, like price and volume sold. Create a correlation matrix visualization.

## TODO normalize columns

### Correlation analysis for conventional avocados in US

In [ ]:
# conventional_total_us_df = total_us_df[total_us_df['type']=='conventional'].loc[:].reset_index(drop=True)

# num_cols=[]
# for col in conventional_total_us_df.columns:
#     if pd.api.types.is_numeric_dtype(conventional_total_us_df[col]):
#         num_cols.append(col)

# corr_matrix_conventional_us=conventional_total_us_df[num_cols].corr()# Create a heatmap of the correlation matrix
# plt.figure(figsize=(10, 8))
# sns.heatmap(corr_matrix_conventional_us, annot=True, cmap=sns.diverging_palette(1, 255, as_cmap=True), fmt=".2f")
# plt.title('Correlation Matrix for whole US only on conventional avocados')
# plt.show()

In [ ]:
scaler = StandardScaler()
numerical_cols = ['AveragePrice','Total Volume','4046','4225','4770','Total Bags','Small Bags','Large Bags','XLarge Bags'] 
df=total_us_df
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

In [ ]:
df = pd.get_dummies(df, columns=['year'])
df

In [ ]:
conventional_total_us_df = df[df['type']=='conventional'].loc[:].reset_index(drop=True)
conventional_total_us_df

In [ ]:
conventional_total_us_df234 = df[df['type']=='conventional'].loc[:].reset_index(drop=True)

num_cols=[]
for col in conventional_total_us_df.columns:
    if pd.api.types.is_numeric_dtype(conventional_total_us_df[col]):
        num_cols.append(col)

corr_matrix_conventional_us1=conventional_total_us_df234[num_cols].corr()# Create a heatmap of the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix_conventional_us1, annot=True, cmap=sns.diverging_palette(1, 255, as_cmap=True), fmt=".2f")
plt.title('Correlation Matrix for whole US only on conventional avocados')
plt.show()

In [ ]:
conventional_total_us_df = total_us_df[total_us_df['type']=='conventional'].loc[:].reset_index(drop=True)

num_cols=[]
for col in conventional_total_us_df.columns:
    if pd.api.types.is_numeric_dtype(conventional_total_us_df[col]):
        num_cols.append(col)

corr_matrix_conventional_us=conventional_total_us_df[num_cols].corr()# Create a heatmap of the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix_conventional_us, annot=True, cmap=sns.diverging_palette(1, 255, as_cmap=True), fmt=".2f")
plt.title('Correlation Matrix for whole US only on conventional avocados')
plt.show()

In [ ]:
correlation_matrix=corr_matrix_conventional_us

# Replace diagonal elements (correlation of 1) with NaN to exclude them
np.fill_diagonal(correlation_matrix.values, np.nan)

# Flatten the correlation matrix
correlation_values = correlation_matrix.values.flatten()

# Sort the correlation values in ascending order and get the indices
sorted_indices = np.argsort(correlation_values)

# Get the 5 lowest correlations
lowest_corr_indices = sorted_indices[:5]
lowest_corr_values = correlation_values[lowest_corr_indices]
lowest_corr_vars = [
    (
        correlation_matrix.columns[i // correlation_matrix.shape[0]],
        correlation_matrix.columns[i % correlation_matrix.shape[0]]
    )
    for i in lowest_corr_indices
]

# Print the 5 lowest and 5 highest correlations
print("\n5 Lowest Correlations:")
for i, (var1, var2) in enumerate(lowest_corr_vars):
    print(f"{i + 1}. Variables: {var1} and {var2}, Correlation Value: {correlation_values[lowest_corr_indices[i]]:.2f}")

# Replace NaN values with a very low value (e.g., -2) to exclude them
correlation_matrix = correlation_matrix.fillna(-2)

# Find the indices of the 5 highest correlations (excluding NaN)
highest_corr_indices = np.unravel_index(np.argsort(correlation_matrix.values, axis=None)[-5:], correlation_matrix.shape)
highest_corr_values = correlation_matrix.values[highest_corr_indices]

# Get the variable pairs for the 5 highest correlations
highest_corr_vars = [
    (
        correlation_matrix.columns[highest_corr_indices[0][i]],
        correlation_matrix.columns[highest_corr_indices[1][i]]
    )
    for i in range(5)
]

# Print the 5 highest correlations
print("\n5 Highest Correlations (Excluding NaN):")
for i, (var1, var2) in enumerate(highest_corr_vars):
    print(f"{i + 1}. Variables: {var1} and {var2}, Correlation Value: {highest_corr_values[i]:.2f}")

In [ ]:
column1 = 'AveragePrice'

# Extract the correlations of 'AveragePrice' with the rest of the columns
correlations_with_column1 = correlation_matrix.loc[column1].drop(column1)

# Get the top 5 highest correlations for 'AveragePrice'
top_5_highest_corr_with_column1 = correlations_with_column1.nlargest(5)
bottom_5_lowest_corr_with_column1 = correlations_with_column1.nsmallest(5)

# Print the top 5 highest correlations for 'AveragePrice'
print(f"\nTop 5 Highest Correlations with {column1}:")
print(top_5_highest_corr_with_column1)

# Print the 5 lowest correlations for 'AveragePrice'
print(f"\n5 Lowest Correlations with {column1}:")
print(bottom_5_lowest_corr_with_column1)

In [ ]:
column2 = 'Total Volume'

# Extract the correlations of 'Total Volume' with the rest of the columns
correlations_with_column2 = correlation_matrix.loc[column2].drop(column2)

# Get the top 5 highest correlations for 'Total Volume'
top_5_highest_corr_with_column2 = correlations_with_column2.nlargest(5)
bottom_5_lowest_corr_with_column2 = correlations_with_column2.nsmallest(5)

# Print the top 5 highest correlations for 'Total Volume'

print(f"\nTop 5 Highest Correlations with {column2}:")
print(top_5_highest_corr_with_column2)

# Print the 5 lowest correlations for 'Total Volume'
print(f"\n5 Lowest Correlations with {column2}:")
print(bottom_5_lowest_corr_with_column2)

In [ ]:
correlation_matrix=corr_matrix_conventional_us

# Replace NaN values with a very low value (e.g., -2) to exclude them
correlation_matrix = correlation_matrix.fillna(-2)

# Find the indices of the 5 highest correlations (excluding NaN)
highest_corr_indices = np.unravel_index(np.argsort(correlation_matrix.values, axis=None)[-5:], correlation_matrix.shape)
highest_corr_values = correlation_matrix.values[highest_corr_indices]

# Get the variable pairs for the 5 highest correlations
highest_corr_vars = [
    (
        correlation_matrix.columns[highest_corr_indices[0][i]],
        correlation_matrix.columns[highest_corr_indices[1][i]]
    )
    for i in range(5)
]

# Print the 5 highest correlations
print("5 Highest Correlations (Excluding NaN):")
for i, (var1, var2) in enumerate(highest_corr_vars):
    print(f"{i + 1}. Variables: {var1} and {var2}, Correlation Value: {highest_corr_values[i]:.2f}")

Based on the heatmap that we can see above there are some correlation that seem important to note.



### Correlation analysis for organic avocados in US

In [ ]:
organic_total_us_df = total_us_df[total_us_df['type']=='organic'].loc[:].reset_index(drop=True)

num_cols=[]
for col in organic_total_us_df.columns:
    if pd.api.types.is_numeric_dtype(organic_total_us_df[col]):
        num_cols.append(col)

corr_matrix_organic_us=organic_total_us_df[num_cols].corr()# Create a heatmap of the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix_organic_us, annot=True, cmap=sns.diverging_palette(1, 255, as_cmap=True), fmt=".2f")
plt.title('Correlation Matrix for whole US only on organic avocados')
plt.show()

In [ ]:
correlation_matrix=corr_matrix_organic_us

# Replace diagonal elements (correlation of 1) with NaN to exclude them
np.fill_diagonal(correlation_matrix.values, np.nan)

# Flatten the correlation matrix
correlation_values = correlation_matrix.values.flatten()

# Sort the correlation values in ascending order and get the indices
sorted_indices = np.argsort(correlation_values)

# Get the 5 lowest correlations
lowest_corr_indices = sorted_indices[:5]
lowest_corr_values = correlation_values[lowest_corr_indices]
lowest_corr_vars = [
    (
        correlation_matrix.columns[i // correlation_matrix.shape[0]],
        correlation_matrix.columns[i % correlation_matrix.shape[0]]
    )
    for i in lowest_corr_indices
]

# Print the 5 lowest and 5 highest correlations
print("\n5 Lowest Correlations:")
for i, (var1, var2) in enumerate(lowest_corr_vars):
    print(f"{i + 1}. Variables: {var1} and {var2}, Correlation Value: {correlation_values[lowest_corr_indices[i]]:.2f}")

# Replace NaN values with a very low value (e.g., -2) to exclude them
correlation_matrix = correlation_matrix.fillna(-2)

# Find the indices of the 5 highest correlations (excluding NaN)
highest_corr_indices = np.unravel_index(np.argsort(correlation_matrix.values, axis=None)[-5:], correlation_matrix.shape)
highest_corr_values = correlation_matrix.values[highest_corr_indices]

# Get the variable pairs for the 5 highest correlations
highest_corr_vars = [
    (
        correlation_matrix.columns[highest_corr_indices[0][i]],
        correlation_matrix.columns[highest_corr_indices[1][i]]
    )
    for i in range(5)
]

# Print the 5 highest correlations
print("\n5 Highest Correlations (Excluding NaN):")
for i, (var1, var2) in enumerate(highest_corr_vars):
    print(f"{i + 1}. Variables: {var1} and {var2}, Correlation Value: {highest_corr_values[i]:.2f}")

In [ ]:
column1 = 'AveragePrice'

# Extract the correlations of 'AveragePrice' with the rest of the columns
correlations_with_column1 = correlation_matrix.loc[column1].drop(column1)

# Get the top 5 highest correlations for 'AveragePrice'
top_5_highest_corr_with_column1 = correlations_with_column1.nlargest(5)
bottom_5_lowest_corr_with_column1 = correlations_with_column1.nsmallest(5)

# Print the top 5 highest correlations for 'AveragePrice'
print(f"\nTop 5 Highest Correlations with {column1}:")
print(top_5_highest_corr_with_column1)

# Print the 5 lowest correlations for 'AveragePrice'
print(f"\n5 Lowest Correlations with {column1}:")
print(bottom_5_lowest_corr_with_column1)

In [ ]:
column2 = 'Total Volume'

# Extract the correlations of 'Total Volume' with the rest of the columns
correlations_with_column2 = correlation_matrix.loc[column2].drop(column2)

# Get the top 5 highest correlations for 'Total Volume'
top_5_highest_corr_with_column2 = correlations_with_column2.nlargest(5)
bottom_5_lowest_corr_with_column2 = correlations_with_column2.nsmallest(5)

# Print the top 5 highest correlations for 'Total Volume'

print(f"\nTop 5 Highest Correlations with {column2}:")
print(top_5_highest_corr_with_column2)

# Print the 5 lowest correlations for 'Total Volume'
print(f"\n5 Lowest Correlations with {column2}:")
print(bottom_5_lowest_corr_with_column2)

## Identify outliers - use statistical methods or visual inspection to find outlier points that could indicate errors.

### Conventional avocados in whole US

In [ ]:
numeric_columns = conventional_total_us_df.select_dtypes(include=[np.number])

# Calculate the Z-scores for the numeric columns
z_scores = np.abs(stats.zscore(numeric_columns))

# Define the threshold for identifying outliers
threshold = 3

# Create a mask to identify outlier rows
outlier_mask = (z_scores > threshold).any(axis=1)

# Get the rows containing outliers
outliers = conventional_total_us_df[outlier_mask]

# Print or further analyze the rows containing outliers
print("Rows containing outliers:")
outliers

#### Average Price

In [ ]:
column_to_check = 'AveragePrice'

# Calculate the Z-scores for the selected columns
z_scores = np.abs(stats.zscore(conventional_total_us_df[column_to_check]))

# Define the threshold for identifying outliers
threshold = 3

# Create a mask to identify outlier rows
outlier_mask = z_scores > threshold

# Get the rows containing outliers in the selected column
outliers = conventional_total_us_df[outlier_mask]

# Print or further analyze the rows containing outliers
print(f"Rows containing outliers in '{column_to_check}':")
outliers

In [ ]:
Q1 = np.percentile(conventional_total_us_df[column_to_check], 25)
Q3 = np.percentile(conventional_total_us_df[column_to_check], 75)
IQR = Q3 - Q1

# Define the lower and upper bounds for identifying outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to get rows with outliers in the selected column
outliers = conventional_total_us_df[(conventional_total_us_df[column_to_check] < lower_bound) | (conventional_total_us_df[column_to_check] > upper_bound)]

# Print or further analyze the rows containing outliers
print(f"Rows containing outliers in '{column_to_check}':")
outliers

In [ ]:
sns.boxplot(x=conventional_total_us_df[column_to_check])
plt.show()

#### Total Volume

In [ ]:
column_to_check = 'Total Volume'

# Calculate the Z-scores for the selected columns
z_scores = np.abs(stats.zscore(conventional_total_us_df[column_to_check]))

# Define the threshold for identifying outliers
threshold = 3

# Create a mask to identify outlier rows
outlier_mask = z_scores > threshold

# Get the rows containing outliers in the selected column
outliers = conventional_total_us_df[outlier_mask]

# Print or further analyze the rows containing outliers
print(f"Rows containing outliers in '{column_to_check}':")
outliers

In [ ]:
Q1 = np.percentile(conventional_total_us_df[column_to_check], 25)
Q3 = np.percentile(conventional_total_us_df[column_to_check], 75)
IQR = Q3 - Q1

# Define the lower and upper bounds for identifying outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to get rows with outliers in the selected column
outliers = conventional_total_us_df[(conventional_total_us_df[column_to_check] < lower_bound) | (conventional_total_us_df[column_to_check] > upper_bound)]

# Print or further analyze the rows containing outliers
print(f"Rows containing outliers in '{column_to_check}':")
outliers

In [ ]:
sns.boxplot(x=conventional_total_us_df[column_to_check])
plt.show()

### Organic avocados in whole US

In [ ]:
numeric_columns = organic_total_us_df.select_dtypes(include=[np.number])

# Calculate the Z-scores for the numeric columns
z_scores = np.abs(stats.zscore(numeric_columns))

# Define the threshold for identifying outliers
threshold = 3

# Create a mask to identify outlier rows
outlier_mask = (z_scores > threshold).any(axis=1)

# Get the rows containing outliers
outliers = organic_total_us_df[outlier_mask]

# Print or further analyze the rows containing outliers
print("Rows containing outliers:")
outliers

#### Average Price

In [ ]:
column_to_check = 'AveragePrice'

# Calculate the Z-scores for the selected columns
z_scores = np.abs(stats.zscore(organic_total_us_df[column_to_check]))

# Define the threshold for identifying outliers
threshold = 3

# Create a mask to identify outlier rows
outlier_mask = z_scores > threshold

# Get the rows containing outliers in the selected column
outliers = organic_total_us_df[outlier_mask]

# Print or further analyze the rows containing outliers
print(f"Rows containing outliers in '{column_to_check}':")
outliers

In [ ]:
Q1 = np.percentile(organic_total_us_df[column_to_check], 25)
Q3 = np.percentile(organic_total_us_df[column_to_check], 75)
IQR = Q3 - Q1

# Define the lower and upper bounds for identifying outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to get rows with outliers in the selected column
outliers = organic_total_us_df[(organic_total_us_df[column_to_check] < lower_bound) | (organic_total_us_df[column_to_check] > upper_bound)]

# Print or further analyze the rows containing outliers
print(f"Rows containing outliers in '{column_to_check}':")
outliers

In [ ]:
sns.boxplot(x=organic_total_us_df[column_to_check])
plt.show()

#### Total Volume

In [ ]:
column_to_check = 'Total Volume'

# Calculate the Z-scores for the selected columns
z_scores = np.abs(stats.zscore(organic_total_us_df[column_to_check]))

# Define the threshold for identifying outliers
threshold = 3

# Create a mask to identify outlier rows
outlier_mask = z_scores > threshold

# Get the rows containing outliers in the selected column
outliers = organic_total_us_df[outlier_mask]

# Print or further analyze the rows containing outliers
print(f"Rows containing outliers in '{column_to_check}':")
outliers

In [ ]:
Q1 = np.percentile(organic_total_us_df[column_to_check], 25)
Q3 = np.percentile(organic_total_us_df[column_to_check], 75)
IQR = Q3 - Q1

# Define the lower and upper bounds for identifying outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to get rows with outliers in the selected column
outliers = organic_total_us_df[(organic_total_us_df[column_to_check] < lower_bound) | (organic_total_us_df[column_to_check] > upper_bound)]

# Print or further analyze the rows containing outliers
print(f"Rows containing outliers in '{column_to_check}':")
outliers

In [ ]:
sns.boxplot(x=organic_total_us_df[column_to_check])
plt.show()

## Check data types - ensure columns have expected data types (string, numeric, datetime, etc.)

## Assess missing data - check for missing/null values and determine strategy for handling them.

# TODO

## Sort the dataframe by the column *Date*

The first step is to sort the dataframe by date in order to have the records in chronological order from earlier observation to latest

In [ ]:
sorted_avocado_df = avocado_df.sort_values(by='Date')

sorted_avocado_df.head()

## Columns of dataset

In [ ]:
sorted_avocado_df['TotalSales'] = sorted_avocado_df['AveragePrice'] * sorted_avocado_df['Total Volume']
sorted_avocado_df['TotalSales'] = sorted_avocado_df['TotalSales'].round(2)

sorted_avocado_df.head()

In [ ]:
column_types = sorted_avocado_df.dtypes
print(column_types)

## Missing values

As we can see on this dataset there are no missing values

In [ ]:
columns = list(sorted_avocado_df)

for column in columns:
    col = Column(column, sorted_avocado_df[column])
    
    print('Column Name : ', col.getName())
    print('Column Null Contents : ', col.getNullContents())

## Types of avocados

In [ ]:
sorted_avocado_df['type'].value_counts()

## Date

In [ ]:
sorted_avocado_df['Date'].value_counts()

## Analysis per region

### Create region objects and append them to an array

In [ ]:
regionValueCounts=sorted_avocado_df['region'].value_counts()

regions={}

for name, volume in regionValueCounts.items():
    region=Region(name,volume, sorted_avocado_df)
    regions[region.getName()]=region

#### Print the details of first region and the head of its dataframe

In [ ]:
regions['Southeast'].details()
regions['Southeast'].getHeadOfDataframe(10)

In [ ]:
regions['Southeast'].PrintStatisticsOfColumns()

### Print plots and statistics of all the regions

### Print plots and statistics of the top 3 regions in sales

#### We will exclude the TotalUS as region to find the top 3

In [ ]:
region_sales = sorted_avocado_df[sorted_avocado_df['region'] != 'TotalUS'].groupby('region')['TotalSales'].sum().reset_index()

# Sort the regions based on total sales in descending order
sorted_regions = region_sales.sort_values(by='TotalSales', ascending=False).reset_index(drop=True)

# Select the top 3 regions with the highest total sales
top_3_regions = sorted_regions.head(3)

print(top_3_regions['region'])

In [ ]:
for name, region in regions.items():
    if name == top_3_regions['region'][0] or name == top_3_regions['region'][1] or name == top_3_regions['region'][2]:
        print(name)
        region.plotColumnAndPrintStatisticsOfIt('TotalSales')
        print("="*20)

### Print plots and statistics of the top 3 regions in Average Price

#### We will exclude the TotalUS as region to find the top 3

In [ ]:
region_sales = sorted_avocado_df[sorted_avocado_df['region'] != 'TotalUS'].groupby('region')['AveragePrice'].mean().reset_index()

# Sort the regions based on total sales in descending order
sorted_regions = region_sales.sort_values(by='AveragePrice', ascending=False).reset_index(drop=True)

# Select the top 3 regions with the highest total sales
top_3_regions = sorted_regions.head(3)

print(top_3_regions)

In [ ]:
for name, region in regions.items():
    if name == top_3_regions['region'][0] or name == top_3_regions['region'][1] or name == top_3_regions['region'][2]:
        print(name)
        region.plotColumnAndPrintStatisticsOfIt('AveragePrice')
        print("="*20)

## Organic type

In [ ]:
organic_df=sorted_avocado_df[sorted_avocado_df.type=='organic']

organic_df.head(10)

### Weekly patterns

### Graphs

## Creation of Model

In [ ]:
# Create a Prophet model
model = Prophet()

In [ ]:
# Preprocess the data
df = region.getConventionalDataframe()[['Date', 'AveragePrice']]
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values(by='Date', inplace=True)
df.columns = ['ds', 'y']
df['ds'] = pd.to_datetime(df['ds'])

df.head()

In [ ]:
# Split the data into training and testing sets
train_size = int(len(df) * 0.8)
train_df =df[:train_size]
test_df = df[train_size:]

In [ ]:
# Fit the model to the training data
model.fit(train_df)

In [ ]:
# Make predictions on the test data
future = model.make_future_dataframe(periods=26, freq='W')
forecast = model.predict(future)

fcst = model.predict(future)
fig = model.plot(fcst)

In [ ]:
# Visualize the results
fig, ax = plt.subplots(figsize=(10, 6))
model.plot(forecast, ax=ax)
ax.plot(test_df.ds, test_df.y, color='black', label='Actual')
ax.legend(loc='upper left')
plt.show()

## Forecast Average Prices for regions

For each of the following columns we will try to forecast on conventional and organic data the future.

### Conventional

In [ ]:
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

cap = 2.25
floor = 0.5

warnings.filterwarnings("ignore")

for name, region in regions.items():
    print(region.getName())
    
    # Create a Prophet model
    model = Prophet(growth = 'logistic')
    
    # Preprocess the data
    df = region.getConventionalDataframe()[['Date', 'AveragePrice']]

    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values(by='Date', inplace=True)
    df.columns = ['ds', 'y']
    df['ds'] = pd.to_datetime(df['ds'])

    df['cap'] = cap
    df['floor'] = floor

    # Split the data into training and testing sets
    train_size = int(len(df) * 0.8)
    train_df =df[:train_size]
    test_df = df[train_size:]

    # Fit the model to the training data
    model.fit(train_df)

    # Make predictions on the test data
    future = model.make_future_dataframe(periods=len(test_df), freq='W')

    future['cap']=cap
    future['floor']=floor

    forecast = model.predict(future)

    fcst = model.predict(future)

    # fig = model.plot(fcst)

    # Visualize the results
    fig, ax = plt.subplots(figsize=(10, 6))
    model.plot(fcst, ax=ax)
    ax.plot(test_df.ds, test_df.y, color='black', label='Actual')
    ax.legend(loc='upper left')

    # Plot the forecast without showing warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        plt.show()

In [ ]:
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

cap = 2.25
floor = 0.5

warnings.filterwarnings("ignore")

region=regions['TotalUS']
print(region.getName())

# Create a Prophet model
model = Prophet(growth = 'logistic')

# Preprocess the data
df = region.getConventionalDataframe()[['Date', 'AveragePrice']]

df['Date'] = pd.to_datetime(df['Date'])
df.sort_values(by='Date', inplace=True)
df.columns = ['ds', 'y']
df['ds'] = pd.to_datetime(df['ds'])

df['cap'] = cap
df['floor'] = floor

# Split the data into training and testing sets
train_size = int(len(df) * 0.8)
train_df =df[:train_size]
test_df = df[train_size:]

# Fit the model to the training data
model.fit(train_df)

# Make predictions on the test data
future = model.make_future_dataframe(periods=len(test_df), freq='W')

future['cap']=cap
future['floor']=floor

forecast = model.predict(future)

fcst = model.predict(future)

# fig = model.plot(fcst)

# Visualize the results
fig, ax = plt.subplots(figsize=(10, 6))
model.plot(fcst, ax=ax)
ax.plot(test_df.ds, test_df.y, color='black', label='Actual')
ax.legend(loc='upper left')

# Plot the forecast without showing warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    plt.show()

### Organic

In [ ]:
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

cap = 2.75
floor = 0.5

warnings.filterwarnings("ignore")

for name, region in regions.items():
    print(region.getName())
    
    # Create a Prophet model
    model = Prophet(growth = 'logistic')
    
    # Preprocess the data
    df = region.getOrganicDataframe()[['Date', 'AveragePrice']]

    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values(by='Date', inplace=True)
    df.columns = ['ds', 'y']
    df['ds'] = pd.to_datetime(df['ds'])

    df['cap'] = cap
    df['floor'] = floor

    # Split the data into training and testing sets
    train_size = int(len(df) * 0.8)
    train_df =df[:train_size]
    test_df = df[train_size:]

    # Fit the model to the training data
    model.fit(train_df)

    # Make predictions on the test data
    future = model.make_future_dataframe(periods=len(test_df), freq='W')

    future['cap']=cap
    future['floor']=floor

    forecast = model.predict(future)

    fcst = model.predict(future)

    # fig = model.plot(fcst)

    # Visualize the results
    fig, ax = plt.subplots(figsize=(10, 6))
    model.plot(fcst, ax=ax)
    ax.plot(test_df.ds, test_df.y, color='black', label='Actual')
    ax.legend(loc='upper left')

    # Plot the forecast without showing warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        plt.show()